In [1]:
import pandas as pd
import gzip
import re

In [2]:
steam_games = pd.read_json('../data/original/steam_games.json.gz',compression='gzip',lines=True)

In [3]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [4]:
#calculo el porcentaje de valores nulos en cada columna (son muchas, mas del 70%)
100*steam_games.isna().sum()/len(steam_games)

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

In [5]:
#Borro todas las filas que estan completamente Nulas
steam_games = steam_games.dropna(how='all')

In [6]:
#vuelvo a calcular el porcentaje de valores nulos en cada columna
100*steam_games.isna().sum()/len(steam_games)

publisher       25.056792
genres          10.216275
app_name         0.006224
title            6.379337
url              0.000000
release_date     6.432239
tags             0.507235
reviews_url      0.006224
specs            2.084954
price            4.285047
early_access     0.000000
id               0.006224
developer       10.266065
dtype: float64

In [7]:
#cantidad de nulos por columna
steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [8]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [9]:
# Completar los generos con las etiquetas de los Tags
steam_games['genres'] = steam_games['tags'].fillna(steam_games['genres'])

In [10]:
steam_games[steam_games['id'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Open World, Batman, Adventure, Stealt...",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [11]:
# columnas que necesito para los endpoints:
# genres  (tags)
# app_name  (title)
# release_date  ()
# id    (url)



In [12]:
steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [13]:
steam_games[['genres', 'app_name','release_date','id']].isna().sum()

genres           139
app_name           2
release_date    2067
id                 2
dtype: int64

In [14]:
#Los registros que tienen valores nulos en "app_name" tambien son nulos en "title" (no se pueden imputar)
steam_games[['app_name','title']][steam_games['app_name'].isna()]

,app_name,title
88384,None,None
90890,None,None


In [15]:
steam_games.dropna(subset=['app_name'], inplace=True)

In [16]:
steam_games[['genres', 'app_name','release_date','id']].isna().sum()

genres           138
app_name           0
release_date    2066
id                 1
dtype: int64

In [17]:
steam_games[['app_name','url','id']][steam_games['id'].isna()]

,app_name,url,id
119271,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,NaN


In [18]:
# El unico valor nulo de "id" se puede imputar con el valor de la "url" 
steam_games['url'][steam_games['id'].isna()].values[0]

'http://store.steampowered.com/app/200260'

In [19]:
# Funcion para extraer el numero de id de la url

def imputar_id(url):
    #buscamos el patron regular en la url
    patron = r"/app/(\d+)"

    #numero de id en url
    num = re.search(patron, url)

    #si es un numero lo retorna, sino sigue nulo
    if num:
        numero = num.group(1)
        return int(numero)
    else:
        return None 

In [20]:
steam_games.loc[steam_games['id'].isna(), 'id'] = steam_games.loc[steam_games['id'].isna(), 'url'].apply(imputar_id)

In [21]:
steam_games[['genres', 'app_name','release_date','id']].isna().sum()

genres           138
app_name           0
release_date    2066
id                 0
dtype: int64

In [22]:
#Verifico que hay 2 id repetidos
steam_games['id'].value_counts()

id
200260.0    2
612880.0    2
530200.0    1
518690.0    1
513460.0    1
           ..
665950.0    1
430094.0    1
676060.0    1
494160.0    1
681550.0    1
Name: count, Length: 32131, dtype: int64

In [23]:
steam_games[['genres', 'app_name','release_date','id']][(steam_games['id']==200260) | (steam_games['id']==612880)]

,genres,app_name,release_date,id
89378,"[Action, Open World, Batman, Adventure, Stealt...",Batman: Arkham City - Game of the Year Edition,2012-09-07,200260.0
102204,"[Action, FPS, Gore, Violent, Alternate History...",Wolfenstein II: The New Colossus,2017-10-26,612880.0
102883,"[Action, FPS, Gore, Violent, Alternate History...",Wolfenstein II: The New Colossus,2017-10-26,612880.0
119271,"[Action, Open World, Batman, Adventure, Stealt...",Batman: Arkham City - Game of the Year Edition,2012-09-07,200260.0


In [24]:
# borro esos registros con valores duplicados en id, app_name y release_date
steam_games.drop_duplicates(subset=['app_name','release_date','id'], inplace=True)

In [25]:
steam_games['app_name'].value_counts()

app_name
Soundtrack                           3
Goro                                 2
Puzzle Blocks                        2
WARZONE                              2
Mars 2030                            2
                                    ..
VR - Killing Town / 杀戮小镇             1
Lionheart                            1
Secret World Legends                 1
Graze Counter Original Soundtrack    1
Maze Run VR                          1
Name: count, Length: 32094, dtype: int64

In [26]:
duplicado = steam_games['app_name'].value_counts() > 2
steam_games[['app_name','release_date','id']][steam_games['app_name'].isin(duplicado[duplicado].index)].sort_values('app_name')

,app_name,release_date,id
93173,Soundtrack,2015-11-30,411110.0
108661,Soundtrack,2016-11-28,561720.0
116400,Soundtrack,2014-10-28,327270.0


In [27]:
duplicado = steam_games['app_name'].value_counts() == 2
steam_games[['app_name','release_date','id']][steam_games['app_name'].isin(duplicado[duplicado].index)].sort_values('app_name')

,app_name,release_date,id
118857,AirMech® Soundtrack,2012-11-13,216173.0
107158,AirMech® Soundtrack,None,599520.0
118489,Aliens: Colonial Marines - Reconnaissance Pack,2013-05-07,219441.0
118488,Aliens: Colonial Marines - Reconnaissance Pack,2013-05-07,224850.0
119713,Alter Ego,2010-08-03,63110.0
...,...,...,...
110009,Ultimate Arena,2016-09-05,436260.0
94892,WARZONE,2016-07-11,494290.0
100232,WARZONE,2017-10-17,712230.0
115981,Warhammer Quest,2015-01-07,329270.0


In [28]:
# borro uno de los registros con valores duplicados en app_name y release_date ()
steam_games.drop_duplicates(subset=['app_name','release_date'], inplace=True)

In [29]:
steam_games[['genres', 'app_name','release_date','id']].isna().sum()

genres           138
app_name           0
release_date    2065
id                 0
dtype: int64

In [30]:
# borro todos los registros nulos que no puedo imputar
steam_games.dropna(subset=['release_date','genres'],inplace=True)

In [31]:
steam_games[['genres', 'app_name','release_date','id']].isna().sum()

genres          0
app_name        0
release_date    0
id              0
dtype: int64

In [32]:
#creo una nueva columna con solo el año usando expresion regular
steam_games['year'] = steam_games['release_date'].str.extract(r'(\d{4})')


In [33]:
#verifico cuantos valores no se lograron imputar el año
steam_games[['genres', 'app_name','release_date','id', 'year']].isna().sum()

genres            0
app_name          0
release_date      0
id                0
year            101
dtype: int64

In [34]:
#observo los valores mas comunes de release_date de los cuales no se pudo imputar el año
steam_games[['genres', 'app_name','release_date','id', 'year']][steam_games['year'].isna()].value_counts('release_date')

release_date
Coming Soon                                                                                   27
TBA                                                                                           16
Coming soon                                                                                    8
TBD                                                                                            6
SOON                                                                                           5
Soon                                                                                           4
coming soon                                                                                    2
Coming Soon!                                                                                   2
When it's done                                                                                 2
Not yet available                                                                              2
To Be Announced  

In [35]:
# borro los regisros sin año
steam_games.dropna(subset=['year'],inplace=True)

In [36]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29824 entries, 88310 to 120443
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     23887 non-null  object 
 1   genres        29824 non-null  object 
 2   app_name      29824 non-null  object 
 3   title         29824 non-null  object 
 4   url           29824 non-null  object 
 5   release_date  29824 non-null  object 
 6   tags          29800 non-null  object 
 7   reviews_url   29824 non-null  object 
 8   specs         29162 non-null  object 
 9   price         28698 non-null  object 
 10  early_access  29824 non-null  float64
 11  id            29824 non-null  float64
 12  developer     28601 non-null  object 
 13  year          29824 non-null  object 
dtypes: float64(2), object(12)
memory usage: 3.4+ MB


In [37]:
steam_games.isna().sum()

publisher       5937
genres             0
app_name           0
title              0
url                0
release_date       0
tags              24
reviews_url        0
specs            662
price           1126
early_access       0
id                 0
developer       1223
year               0
dtype: int64

In [38]:
#columnas que voy a usar para los endpoints
steam_games[['genres', 'app_name','id', 'year']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 29824 entries, 88310 to 120443
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genres    29824 non-null  object 
 1   app_name  29824 non-null  object 
 2   id        29824 non-null  float64
 3   year      29824 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.1+ MB


In [39]:
steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer', 'year'],
      dtype='object')

In [40]:
columnas_borrar=['title', 'url', 'tags', 'reviews_url', 'specs', 'early_access']
steam_games.drop(columns=columnas_borrar, inplace=True)

In [41]:
#exporto la data de user_review en formato CSV comprimido con la info basica que necesito.
with gzip.open('../data/limpio/steam_games.csv.gz', 'wb') as file:
    steam_games.to_csv(file, index=False, encoding='utf-8')